# Preprocessing Data

In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.image as img
from tqdm import tqdm
import os, cv2

TRAIN_DIR = 'C:\\Users\\rocke\\URE18\\Images'
TEST_DIR = 'C:\\Users\\rocke\\URE18\\Test Images'
TRAIN_LABEL_DIR = 'Data_Entry_2017.csv'
IMG_SIZE = 256

In [ ]:
# Save baseline model architecture and weights
def save_base(model_name):
    model_str = str(input("Save model as: "))

    # serialize model to JSON
    model_name = baseline.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    baseline.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Save Comparison model
def save_model(model_name):
    model_str = str(input("Save model as: "))

    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Load model architecture and weights NOTE: must compile again
def load_model():
    model_str = str(input("Name of model to load: "))

    # load json and create model
    json_file = open('CNN Models/' + model_str + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("CNN Models/" + model_str + ".h5")
    print("Loaded " + model_str + " and weights from CNN Models folder")
    
    return loaded_model

# Write history object to a file using pickle
def save_history(model_name):
    hist_str = str(input("Save history as: "))

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
# Load history object
def load_history():
    hist_str = str(input("Name of history to load: "))

    loaded_history = pickle.load(open('Training Histories/' + hist_str + '.p', 'rb'))
    
    print("Loaded " + hist_str + " from Training Histories folder")
    
    return loaded_history

In [ ]:
# One Hot Encode for Multiple Labels
'''
from sklearn.preprocessing import MultiLabelBinarizer

read_data = pd.read_csv(TRAIN_LABEL_DIR)
split_labels = [items.split('|') for items in read_data.Finding_Labels]
one_hot = MultiLabelBinarizer()
onehot_labels = one_hot.fit_transform(split_labels)
test_labels = onehot_labels[:11212]
training_labels = onehot_labels[11212:]

pickle.dump(training_labels, open('training_labels.p', 'wb'))
pickle.dump(test_labels, open('test_labels.p', 'wb'))

def create_batch(directory):
    img_array = []
    for img in tqdm(os.listdir(directory)):
        path = os.path.join(directory, img)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_GRAYSCALE), (IMG_SIZE, IMG_SIZE))
        img_array.append(img)
    return img_array

training_img = create_batch(TRAIN_DIR)
training_img = np.array(training_img)
training_img = np.expand_dims(training_img, axis = 3)

test_img = create_batch(TEST_DIR)
test_img = np.array(test_img)
test_img = np.expand_dims(test_img, axis = 3)

# Save Processed Images
#pickle.dump(training_img[:int(len(training_img)/2)], open('training_img_1.p', 'wb'))
#pickle.dump(training_img[int(len(training_img)/2):], open('training_img_2.p', 'wb'))
#pickle.dump(test_img, open('pickle_test_img.p', 'wb'))
'''

### Load Processed Images

In [ ]:
# Load with pickle instead of processing images again
training_img_1 = pickle.load(open('training_img_1.p', 'rb'))
training_img_2 = pickle.load(open('training_img_2.p', 'rb'))
training_img = np.append(training_img_1, training_img_2, axis = 0)

test_img = pickle.load(open('pickle_test_img.p', 'rb'))

training_labels = pickle.load(open('training_labels.p', 'rb'))
test_labels = pickle.load(open('test_labels.p', 'rb'))

In [ ]:
print(training_img.shape, 
     training_labels.shape,
     test_img.shape,
     test_labels.shape)

# Neural Network

In [ ]:
import keras
from keras import models, optimizers, layers
from keras.models import model_from_json
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)

In [ ]:
baseline = models.Sequential()
baseline.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
baseline.add(layers.MaxPooling2D((2, 2)))
baseline.add(layers.Conv2D(64, (3, 3), activation='relu'))
baseline.add(layers.MaxPooling2D((2, 2)))
baseline.add(layers.Conv2D(64, (3, 3), activation='relu'))
baseline.add(layers.MaxPooling2D((2, 2)))

baseline.add(layers.Flatten())
baseline.add(layers.Dense(64, activation='relu'))
baseline.add(layers.Dense(15, activation='sigmoid'))

#baseline.summary()

In [ ]:
baseline.compile(optimizer = optimizers.RMSprop(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
base_history = baseline.fit(training_img, training_labels, epochs = 40, validation_split = (1 / 9), batch_size = 128, verbose = 1)

In [ ]:
save_history(base_history)

#### Save Base Model

In [ ]:
save_base(base_history)

In [ ]:
baseline_test = baseline.evaluate(test_img, test_labels)
print(baseline_test)

In [ ]:
import matplotlib.pyplot as plt

acc = base_history.history['acc']
val_acc = base_history.history['val_acc']
loss = base_history.history['loss']
val_loss = base_history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

# Comparison Model

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.summary()

In [ ]:
model.compile(optimizer = optimizers.RMSprop(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model_4C_4MP_1D = model.fit(training_img, training_labels, epochs = 20, validation_split = (1 / 9), batch_size = 128, verbose = 1)

In [ ]:
model_test = model.evaluate(test_img, test_labels)

In [ ]:
print(model_test)

In [ ]:
# Write history object to a file using pickle
def save_history(model_name):
    hist_str = str(input("Save history as: "))

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")

In [ ]:
save_history(model_4C_4MP_1D)

### Plot Results Compared To Baseline

In [ ]:
base_acc = bases_loaded.history['val_acc']
model_acc = model_4C_4MP_1D.history['val_acc']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, base_acc, 'b', label='Base Acc')
plt.plot(epochs, model_acc, 'r', label='Model Acc')
plt.title('Comparison of Validation Accuracies')
plt.legend()
plt.show()

In [ ]:
base_loss = bases_loaded.history['val_loss']
model_loss = model_4C_4MP_1D.history['val_loss']
plt.plot(epochs, base_loss, 'b', label='Base loss')
plt.plot(epochs, model_loss, 'r', label='Model loss')
plt.title('Comparison of Validation Losses')
plt.legend()
plt.show()